<a href="https://colab.research.google.com/github/KelvinYCH5354/Linear-Regression/blob/main/colab_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

**Goal**: predict the PM2.5 concentration for the next 10 hours based on the previous 9 hours' data of 18 features (including PM2.5).

If there are any questions, please feel free to email the teaching assistant mailbox: xiang.hao@connect.polyu.hk.

_Modified from Lee Hong Yee's tutorials_

## Load `train.csv`

The data in train.csv consists of 20 days per month, 24 hours per day, and 18 features per hour for each of the 12 months.

In [ ]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive

# Download `data.zip` and extract it.
!gdown '1EDo3Q8TTKKPLU1eWE9ZBAOIJUxIAiS9i' --output data.zip
!unzip data.zip

data = pd.read_csv('./train.csv', encoding = 'utf-8')

Downloading...
From: https://drive.google.com/uc?id=1EDo3Q8TTKKPLU1eWE9ZBAOIJUxIAiS9i
To: /content/data.zip
100% 166k/166k [00:00<00:00, 85.5MB/s]
Archive:  data.zip
  inflating: test.csv                
  inflating: train.csv               


## Preprocessing
Extract the required values and fill the 'RAINFALL' column with 0 (NR: No Rain).

Also, if you want to repeat the execution of this code in Colab, please start from the beginning (rerun the above code) to avoid getting unexpected results. (If you write the code yourself, you may not encounter this issue, but in Colab, repeating this code will keep taking data down. That is, the first time you take data from column 3 and beyond, the second time you take the data from the first time and drop the first three columns, and so on.)

In [ ]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

## Extract Features (1)

![image](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![image](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

Reorganize the original $4320 \times 18$ data into 12 sets of 18 (features) $×$ 480 (hours) data according to each month. The image descriptions show two images that cannot be translated without context.

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

## Extract Features (2)
![alt text](https://drive.google.com/uc?id=1m2cDxtOURX4w3JernurpmuTgwc3sjREB)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

here are 480 hours per month, with one data point generated every 9 hours. There will be 471 data points per month, resulting in a total of $471 \times 12$ data points. Each data point has $9 \times 18$ features (18 features per hour $\times$ 9 hours).
There will be $471 \times 12$ corresponding targets (PM2.5 at the 10th hour).

In [ ]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:  # there must be 9 hours remaining
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

## Normalize


In [ ]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

## Split Training Data Into "train_set" and "validation_set"

This section provides a simple demonstration for the second and third questions in the homework report, generating a train_set for training and a validation_set that will not be included in the training process but only used for verification.

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

## Training

![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(Different from the previous figure: the code below uses Root Mean Square Error)

Because of the existence of the constant term, an additional column is needed for the dimension (dim); the eps term is added to avoid the denominator of adagrad being 0.

Each dimension (dim) corresponds to its own gradient and weight (w), and is learned through iterations (iter_time).

In [ ]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

## Testing
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

Load the test data and preprocess and feature extract it in a similar way to the training data, so that the test data forms 240 data points with dimensions of 18 * 9 + 1.

In [ ]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

## Prediction

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

With the weight and test data, the target can be predicted.

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

## Save Prediction to CSV File


In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

Related reference can refer to:

Adagrad:
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705

RMSprop:
https://www.youtube.com/watch?v=5Yt-obwvMHI

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs


The above print part is mainly to see the presentation of data and results, and it doesn't hurt to remove it. In addition, in your own Linux system, you can replace the hard-coded part of the file with the use of sys.argv (you can enter the file and file location yourself in the terminal).

Finally, you can go beyond the baseline by adjusting the learning rate, iter_time (number of iterations), the number of features used (how many hours to take, which feature fields to take), and even different models.